In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

base_url = r'/home/amit/DataScienceProject/PredictDroughts/'

In [12]:
df = pd.read_csv(base_url + 'files/merge.csv', index_col=0)
df

,Week,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,...,TS,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,LEVEL
0,2000-01-04,Alabama,AL,1,-86.902298,32.318231,12.35,2.14,6.84,15.39,...,12.18,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97,1
1,2000-01-11,Alabama,AL,1,-86.902298,32.318231,0.97,1.18,7.09,13.70,...,11.15,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89,1
2,2000-01-18,Alabama,AL,1,-86.902298,32.318231,1.80,1.59,9.91,6.50,...,14.94,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94,1
3,2000-01-25,Alabama,AL,1,-86.902298,32.318231,0.00,2.20,2.15,10.89,...,0.24,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14,1
4,2000-02-01,Alabama,AL,1,-86.902298,32.318231,0.01,0.73,3.38,15.38,...,2.15,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,2021-05-18,Wyoming,WY,56,-107.290284,43.075968,0.00,0.47,6.54,17.38,...,17.56,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45,1
58236,2021-05-25,Wyoming,WY,56,-107.290284,43.075968,0.11,0.95,5.07,16.60,...,13.43,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50,1
58237,2021-06-01,Wyoming,WY,56,-107.290284,43.075968,1.19,1.75,7.33,12.02,...,17.39,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27,1
58238,2021-06-08,Wyoming,WY,56,-107.290284,43.075968,0.00,0.49,6.30,17.62,...,21.32,13.17,14.71,11.86,11.36,80.92,3.91,12.01,12.41,1


In [13]:
df['Week'] = pd.to_datetime(df['Week'])
set_year = df['Week'].dt.year.to_list()
set_month = df['Week'].dt.month.to_list()
set_day = df['Week'].dt.day.to_list()
df.insert(0, "Year", set_year, None)
df.insert(1, "Month", set_month, None)
df.insert(2, "Day", set_day, None)
df = df.drop(columns='Week', axis=1)
df

,Year,Month,Day,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,...,TS,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,LEVEL
0,2000,1,4,Alabama,AL,1,-86.902298,32.318231,12.35,2.14,...,12.18,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97,1
1,2000,1,11,Alabama,AL,1,-86.902298,32.318231,0.97,1.18,...,11.15,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89,1
2,2000,1,18,Alabama,AL,1,-86.902298,32.318231,1.80,1.59,...,14.94,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94,1
3,2000,1,25,Alabama,AL,1,-86.902298,32.318231,0.00,2.20,...,0.24,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14,1
4,2000,2,1,Alabama,AL,1,-86.902298,32.318231,0.01,0.73,...,2.15,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,2021,5,18,Wyoming,WY,56,-107.290284,43.075968,0.00,0.47,...,17.56,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45,1
58236,2021,5,25,Wyoming,WY,56,-107.290284,43.075968,0.11,0.95,...,13.43,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50,1
58237,2021,6,1,Wyoming,WY,56,-107.290284,43.075968,1.19,1.75,...,17.39,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27,1
58238,2021,6,8,Wyoming,WY,56,-107.290284,43.075968,0.00,0.49,...,21.32,13.17,14.71,11.86,11.36,80.92,3.91,12.01,12.41,1


In [14]:
encoding_columns = pd.get_dummies(df['LEVEL'], prefix="level")
df = df.join(encoding_columns)
df

,Year,Month,Day,State,Postal Code,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,...,PS,T2MDEW,T2M_MIN,T2MWET,LEVEL,level_0,level_1,level_2,level_3,level_4
0,2000,1,4,Alabama,AL,1,-86.902298,32.318231,12.35,2.14,...,100.72,6.95,2.85,6.97,1,0,1,0,0,0
1,2000,1,11,Alabama,AL,1,-86.902298,32.318231,0.97,1.18,...,100.83,8.89,5.91,8.89,1,0,1,0,0,0
2,2000,1,18,Alabama,AL,1,-86.902298,32.318231,1.80,1.59,...,100.82,13.94,11.96,13.94,1,0,1,0,0,0
3,2000,1,25,Alabama,AL,1,-86.902298,32.318231,0.00,2.20,...,100.53,-7.34,-4.39,-7.14,1,0,1,0,0,0
4,2000,2,1,Alabama,AL,1,-86.902298,32.318231,0.01,0.73,...,101.43,-1.58,-3.99,-1.55,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,2021,5,18,Wyoming,WY,56,-107.290284,43.075968,0.00,0.47,...,80.75,4.49,6.61,10.45,1,0,1,0,0,0
58236,2021,5,25,Wyoming,WY,56,-107.290284,43.075968,0.11,0.95,...,81.35,1.03,3.02,6.50,1,0,1,0,0,0
58237,2021,6,1,Wyoming,WY,56,-107.290284,43.075968,1.19,1.75,...,81.70,6.35,9.74,11.27,1,0,1,0,0,0
58238,2021,6,8,Wyoming,WY,56,-107.290284,43.075968,0.00,0.49,...,80.92,3.91,12.01,12.41,1,0,1,0,0,0


In [15]:
X = df[df.columns[(df.columns != 'LEVEL') & (df.columns != 'State') & (df.columns != 'Postal Code')]]
X

,Year,Month,Day,Fips,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,...,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,level_0,level_1,level_2,level_3,level_4
0,2000,1,4,1,-86.902298,32.318231,12.35,2.14,6.84,15.39,...,3.30,100.72,6.95,2.85,6.97,0,1,0,0,0
1,2000,1,11,1,-86.902298,32.318231,0.97,1.18,7.09,13.70,...,1.33,100.83,8.89,5.91,8.89,0,1,0,0,0
2,2000,1,18,1,-86.902298,32.318231,1.80,1.59,9.91,6.50,...,2.28,100.82,13.94,11.96,13.94,0,1,0,0,0
3,2000,1,25,1,-86.902298,32.318231,0.00,2.20,2.15,10.89,...,2.16,100.53,-7.34,-4.39,-7.14,0,1,0,0,0
4,2000,2,1,1,-86.902298,32.318231,0.01,0.73,3.38,15.38,...,1.69,101.43,-1.58,-3.99,-1.55,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,2021,5,18,56,-107.290284,43.075968,0.00,0.47,6.54,17.38,...,2.89,80.75,4.49,6.61,10.45,0,1,0,0,0
58236,2021,5,25,56,-107.290284,43.075968,0.11,0.95,5.07,16.60,...,5.50,81.35,1.03,3.02,6.50,0,1,0,0,0
58237,2021,6,1,56,-107.290284,43.075968,1.19,1.75,7.33,12.02,...,2.58,81.70,6.35,9.74,11.27,0,1,0,0,0
58238,2021,6,8,56,-107.290284,43.075968,0.00,0.49,6.30,17.62,...,11.36,80.92,3.91,12.01,12.41,0,1,0,0,0


In [16]:
y = df[['LEVEL']]
y

,LEVEL
0,1
1,1
2,1
3,1
4,1
...,...
58235,1
58236,1
58237,1
58238,1


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [18]:
def ML_Algo(model):
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test) * 100
    return score

In [19]:
score_list = []
models = ["LogisticRegression", "KNeighborsClassifier", "DecisionTreeClassifier", "RandomForestClassifier", "SVC"]

In [20]:
lg = LogisticRegression()
lg_score = ML_Algo(lg)
score_list.append(lg_score)

/home/amit/projects/regstrtion_form/env/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/amit/projects/regstrtion_form/env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
knn = KNeighborsClassifier(n_neighbors=5)
knn_score = ML_Algo(knn)
score_list.append(knn_score)

<ipython-input-18-cfba63d19b1a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  model.fit(X_train, y_train)


In [22]:
dtc = DecisionTreeClassifier()
dtc_score = ML_Algo(dtc) 
score_list.append(dtc_score)

In [23]:
rfc = RandomForestClassifier()
rfc_score = ML_Algo(rfc)
score_list.append(rfc_score)

<ipython-input-18-cfba63d19b1a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


In [24]:
svc = SVC()
svc_score = ML_Algo(svc)
score_list.append(svc_score)

/home/amit/projects/regstrtion_form/env/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [25]:
data = {
    "Model": models,
    "Score": score_list
}

In [26]:
score_df = pd.DataFrame(data)
score_df

,Model,Score
0,LogisticRegression,70.890568
1,KNeighborsClassifier,71.096612
2,DecisionTreeClassifier,100.000000
3,RandomForestClassifier,100.000000
4,SVC,71.440018
